# Basic RAG over an existing vector database

In [3]:
!pip install langchain langchain_ollama
!pip install chromadb sentence-transformers langchain_huggingface langchain_chroma

  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
  Using cached chroma_hnswlib-0.7.6.tar.gz (32 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached fastapi-0.115.6-py3-none-any.whl.metadata (27 kB)
  Using cached posthog-3.7.4-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached onnxruntime-1.20.1-cp313-cp313-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
  Using cached opentelemetry_api-1.29.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.29.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.50b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_sdk-1.29.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached tokeni

Select **model** for embeddings. 

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/home/bigdata/miniconda3/envs/Trabajo_BIDA/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Access existing database

In [5]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="some_facts",
    embedding_function=embeddings,
    persist_directory="./chroma_some_facts",  # Where to save data locally, remove if not necessary
)

Create retriever

In [6]:
retriever = vector_store.as_retriever()

Make a search in the vectorstor (only for testing purposes)

In [7]:
question = "Can I steal a bank?"
docs = vector_store.similarity_search(question)
len(docs)

0

In [8]:
docs[0]

IndexError: list index out of range

RAG Chain

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "llama3.2"
model_local = ChatOllama(model=ollama_llm)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

In [ ]:
# Question
chain.invoke("Can I steal a bank?")